<a href="https://colab.research.google.com/github/sipocz/Alarm/blob/main/bottle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget https://github.com/sipocz/CabPrice/raw/B20210507/TRAIN.csv

--2021-05-31 17:25:20--  https://github.com/sipocz/CabPrice/raw/B20210507/TRAIN.csv
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/CabPrice/B20210507/TRAIN.csv [following]
--2021-05-31 17:25:20--  https://raw.githubusercontent.com/sipocz/CabPrice/B20210507/TRAIN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7524003 (7.2M) [text/plain]
Saving to: ‘TRAIN.csv.1’

TRAIN.csv.1         100%[===================>]   7.17M  13.7MB/s    in 0.5s    

2021-05-31 17:25:21 (13.7 MB/s) - ‘TRAIN.csv.1’ saved [7524003/7524003]



In [5]:
def DataPipeLine(df, startup=True):
    #outlierek 
    if startup:
        df["price_per"]=df.fare/df.surge_multiplier/df.distance
        df.drop(df[df['price_per'] > 373].index, inplace = True)
        #df.drop(df[df.cab_provider=="Lyft"].index, inplace=True)
    
    df["sec"]=(df.time_stamp/1000) % (24*3600)
   
    seconds_in_day = 24*60*60
    df['sin_time'] = (np.sin(2*np.pi*df.sec/seconds_in_day))
    df['cos_time'] = (np.cos(2*np.pi*df.sec/seconds_in_day))

    sources=pd.get_dummies(df["source"],prefix='source',drop_first=False)
    dest=pd.get_dummies(df["destination"],prefix='dest',drop_first=False)
    cab=pd.get_dummies(df["cab_provider"],prefix='provider',drop_first=False)
    
    a=df.time_stamp
    df["DateTime"]=a.apply( lambda x:pd.Timestamp(x,unit="ms"))
    df["move_sum"]=df.rolling("5s",on='DateTime' ).distance.mean()
    #df["dow"]=df["DateTime"].dt.dayofweek
    
    #dow=pd.get_dummies(df["dow"],prefix='dow',drop_first=False)


    sources=pd.get_dummies(df["source"],prefix='source',drop_first=False)
    dest=pd.get_dummies(df["destination"],prefix='dest',drop_first=False)
    cab=pd.get_dummies(df["cab_provider"],prefix='provider',drop_first=False)
    cabtype=pd.get_dummies(df["cab_type"],prefix='cab_type',drop_first=False)

    new = pd.DataFrame([df.DateTime, sources.columns, dest.columns])


    Xret = pd.DataFrame([
                    
                    cab['provider_Lyft'],
                    cab['provider_Uber'],
                    sources['source_Back Bay'],sources['source_Beacon Hill'],sources['source_Boston University'],
                    sources['source_Fenway'],sources['source_Financial District'],sources['source_Haymarket Square'],
                    sources['source_North End'],sources['source_North Station'],sources['source_Northeastern University'],
                    sources['source_South Station'],sources['source_Theatre District'],sources['source_West End'],
                     
                    dest['dest_Back Bay'],dest['dest_Beacon Hill'],dest['dest_Boston University'],
                    dest['dest_Fenway'],dest['dest_Financial District'],dest['dest_Haymarket Square'],
                    dest['dest_North End'],dest['dest_North Station'],dest['dest_Northeastern University'],
                    dest['dest_South Station'],dest['dest_Theatre District'],dest['dest_West End'],

                    cabtype['cab_type_Black'],cabtype['cab_type_Black SUV'],cabtype['cab_type_Lux'],cabtype['cab_type_Lux Black'],
                    cabtype['cab_type_Lux Black XL'],cabtype['cab_type_Lyft'],cabtype['cab_type_Lyft XL'],cabtype['cab_type_Shared'],
                    cabtype['cab_type_UberPool'],cabtype['cab_type_UberX'],cabtype['cab_type_UberXL'],cabtype['cab_type_WAV'],
                    #dow["dow_0"],dow["dow_1"],dow["dow_2"],



                    df["distance"] /10,
                    df["surge_multiplier"]/3,
                    df["move_sum"],
                    (df["sin_time"]/2)+0.5,
                    (df["cos_time"]/2)+0.5
                    #df.fare/df["surge_multiplier"]/df["distance"],
                    #df.fare
                    ]).transpose()
    return(Xret)



In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("TRAIN.csv")
X_train=DataPipeLine(df)
y_train=df["fare"]/df["fare"].max()

In [8]:
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences

In [12]:
class encodecode():
    
    opt = tf.keras.optimizers.Adamax(learning_rate=0.0001)
    rmse = tf.keras.metrics.RootMeanSquaredError()
    initializer = tf.keras.initializers.LecunNormal()
    
    def __init__(self,X_train, y_train):
        self.X_train=X_train
        self.y_train=y_train
        self.input_len=len(X_train.columns)
    
    def encoder(self):
        self.input1=Input(shape=(self.input_len,),)
        
        e1=Dense(100,activation="sigmoid",kernel_initializer=self.initializer)(self.input1)
        e2=Dense( 50,activation="sigmoid",kernel_initializer=self.initializer)(e1)
        self.bottle=Dense(1,activation="sigmoid",kernel_initializer=self.initializer)(e2)
        
    def decoder(self):
        d1=Dense( 50,activation="sigmoid",kernel_initializer=self.initializer)(self.bottle)
        d2=Dense(100,activation="sigmoid",kernel_initializer=self.initializer)(d1)
        d3=Dense(self.input_len,activation="sigmoid",kernel_initializer=self.initializer)(d2)
        self.pred=d3

    def calculate(self):
        #clear_session()
        self.encoder()
        self.decoder()
        self.model = Model(inputs=self.input1, outputs=self.pred,)
        self.model.summary()
        self.model.compile(
                        loss="mean_squared_error",
                        optimizer=self.opt,
                        metrics=[self.rmse]
                          )
        
    def modellfit(self):
        self.history = self.model.fit( 
                        self.X_train[:],self.y_train[:], 
                        epochs=555, 
                        batch_size=16,validation_split=0.2,
                        verbose=1,
                        shuffle=False,
                        )



In [10]:
X_train.head()

,provider_Lyft,provider_Uber,source_Back Bay,source_Beacon Hill,source_Boston University,source_Fenway,source_Financial District,source_Haymarket Square,source_North End,source_North Station,source_Northeastern University,source_South Station,source_Theatre District,source_West End,dest_Back Bay,dest_Beacon Hill,dest_Boston University,dest_Fenway,dest_Financial District,dest_Haymarket Square,dest_North End,dest_North Station,dest_Northeastern University,dest_South Station,dest_Theatre District,dest_West End,cab_type_Black,cab_type_Black SUV,cab_type_Lux,cab_type_Lux Black,cab_type_Lux Black XL,cab_type_Lyft,cab_type_Lyft XL,cab_type_Shared,cab_type_UberPool,cab_type_UberX,cab_type_UberXL,cab_type_WAV,distance,surge_multiplier,move_sum,sin_time,cos_time
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303,0.333333,3.030000,0.91054,0.785407
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130,0.333333,2.165000,0.91054,0.785407
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.271,0.333333,2.346667,0.91054,0.785407
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243,0.333333,2.367500,0.91054,0.785407
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.271,0.333333,2.436000,0.91054,0.785407


In [13]:

deccer=encodecode(X_train,X_train)
deccer.X_train.head()
deccer.encoder()
deccer.decoder()

deccer.calculate()
deccer.model.summary()

deccer.modellfit()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
dense_18 (Dense)             (None, 100)               4400      
_________________________________________________________________
dense_19 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 51        
_________________________________________________________________
dense_21 (Dense)             (None, 50)                100       
_________________________________________________________________
dense_22 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_23 (Dense)             (None, 43)                4343

KeyboardInterrupt: ignored

In [ ]:
deccer.modellfit()

In [ ]:
list(deccer.bottle[0,:])